In [16]:
import pandas as pd
import re
from nltk.tokenize import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm') # Load spacy model
from bs4 import BeautifulSoup
import spacy
import unidecode
#from word2number import w2n
#import contractions
import os

In [17]:
STOCK_NEWS_DATASET_FILE = r"C:\Users\Elske\NLP local\aapl_us_equities_news_processed.csv"

In [18]:
df_content = pd.read_csv(STOCK_NEWS_DATASET_FILE)

In [19]:
len(df_content)

1740

In [21]:
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

In [22]:
def clean_txt(txt):

    """
    Input: Text

    Remove every non-alphanumeric characters
    Lowercase every word except 'AAPL'

    Output: Text
    """

    # Onderstaande twee regels van https://towardsdatascience.com/nlp-text-preprocessing-steps-tools-and-examples-94c91ce5d30
    txt = re.sub(r'\W+', ' ', txt) # matches any non-word character (equivalent to [^a-zA-Z0-9_])
    txt = re.sub(r'[^a-zA-Z]', ' ', txt) # [^a-zA-Z] means any character that IS NOT a-z OR A-Z
    
    words = [word.lower() for word in txt.split(' ')] # Capslocks verwijderen voor alle woorden behalve AAPL
    txt_temp = " ".join(words) # Join losse woorden weer samen tot 1 text    

    return txt_temp

In [23]:
def remove_stopwords(text):

    """
    Input: Text
    Remove stopwords and lemmatize text.
    Output: Text
    """

    return " ".join(token.lemma_ for token in nlp(text) if not token.is_stop)

In [24]:
df_content['stripped_html'] = df_content.apply(lambda row : strip_html_tags(row['text']), axis = 1)
df_content['lower_cased'] = df_content.apply(lambda row : clean_txt(row['stripped_html']), axis = 1)

In [26]:
# Execution functions
df_content['removed_stopwords'] = df_content.apply(lambda row : remove_stopwords(row['lower_cased']), axis = 1)

In [27]:
sum(df_content['stripped_html'] != df_content['text']) # Geen HTML text

0

In [28]:
os.chdir(r'C:\Users\Elske\NLP local')
pd.DataFrame(df_content).to_csv('Preprocessed_Newsarticles_6okt2022.csv')

### NOTES ###

In [29]:
# Lemma even testen
token = 'riding'
[token.lemma_ for token in nlp(token)]

['ride']

In [30]:
# New stop words list 
customize_stop_words = [
    ':)'
]

# Mark them as stop words
for w in customize_stop_words:
    nlp.vocab[w].is_stop = True

### Alert zijn op volgende punten:
* US wordt getokenized als u en s
